In [103]:
# Read input
import numpy as np
import itertools
from tqdm import tqdm
import pickle

def map2int(ls):
    return [int(st) for st in ls]

def read_int_list(f):
    return map2int(f.readline().split(' '))

def cal_distance(x0, y0, x1, y1):
    return abs(x1-x0) + abs(y1-y0)

def cal_end2start(ride1, ride2):
    return cal_distance(ride1['x'], ride1['y'], ride2['a'], ride2['b'])

def possible_follow(ride1, ride2, end2start):
    earliest_finish = ride1['start_els'] + ride1['distance']
    return (earliest_finish + end2start < ride2['critical_start'])

def cal_loss_waiting_time(waiting_time):
    return waiting_time * coeff_waiting_loss

def cal_loss_bonus(distance):
    return distance * coeff_bonus_loss

def is_possible(ride2, end2start, cur_t):
    return (cur_t + end2start < ride2['critical_start'])
        
def cal_reward(ride1, ride2, cur_t, end2start):
    bonus = 0
    # Bonus
    if cur_t + end2start <= ride2['start_els']:
        bonus = B
    dist = ride2['distance']
    
    waiting_time = end2start
    if cur_t + end2start < ride2['start_els']:
        waiting_time += ride2['start_els'] - (cur_t + end2start)
        
    loss_waiting = cal_loss_waiting_time(waiting_time)
    loss_bonus = cal_loss_bonus(dist)
    return (bonus, bonus + dist, waiting_time, dist, bonus + dist - loss_waiting - loss_bonus)

#fname = 'b_should_be_easy.in'
#fout = 'b_should_be_easy.out'
#fname = 'c_no_hurry.in'
#fout = 'c_no_hurry.out'
fname = 'd_metropolis.in'
fout = 'd_metropolis.out'
#fname = 'e_high_bonus.in'
#fout = 'e_high_bonus.out'        

In [ ]:
with open(fname) as f:
    R, C, F, N, B, T = read_int_list(f)
    rides = []
    #Rend2start = np.full((N+1, N+1), -np.inf, dtype=float)   
    for i_ride in range(N):
        x0, y0, x1, y1, start_els, finish = read_int_list(f)
        distance = cal_distance(x0, y0, x1, y1)
        rides.append({'a': x0, 'b':y0, 'x':x1, 'y':y1, 
                      'distance': distance, 'critical_start': finish-distance,
                      'start_els':start_els, 'finish':finish})
        
    # Ride N+1 is the starting point
    rides.append({'a':0, 'b':0, 'x':0, 'y':0,
                  'distance': 0, 'critical_start': 0,
                  'start_els': 0, 'finish': 0})
    
    # Precalculate rides that can follow R
    R_links = {}
    End2Start = np.zeros((N+1, N+1), dtype=int)
    for i_ride1 in tqdm(range(N)):
        ride1 = rides[i_ride1]
        R_links[str(i_ride1)] = list()
        for i_ride2 in range(N):
            if i_ride1 == i_ride2: continue
            ride1, ride2 = rides[i_ride1], rides[i_ride2]
            end2start = cal_end2start(ride1, ride2)
            End2Start[i_ride1, i_ride2] = end2start
            if possible_follow(ride1, ride2, end2start):
                R_links[str(i_ride1)].append(i_ride2)
            #print(len(R_links[i_ride1]))
    i_ride1 = N
    ride1 = rides[i_ride1]
    R_links[str(i_ride1)] = list(range(N))
    for i_ride2 in range(N):
        ride2 = rides[i_ride2]
        End2Start[i_ride1, i_ride2] = cal_end2start(ride1, ride2)
        
pkfname = fname + 'pk'
with open(pkfname, 'wb') as f:
    pickle.dump((R_links, End2Start, rides, B, N, F ), f)


In [104]:
pkfname = fname + 'pk'
with open(pkfname, 'rb') as f:
    R_links, End2Start, rides, B, N, F = pickle.load(f)

In [105]:
avg_dist = sum([ride['distance'] for ride in rides])/len(rides)
avg_end2start = np.mean(End2Start)

In [ ]:
# Deep First Search
coeff_waiting_loss = (B + avg_dist)/(avg_dist + avg_end2start)
coeff_bonus_loss = B/(avg_dist + avg_end2start)
coeff_exp = 0.95

VclRides = [list() for _ in range(F)]
selected_rides = set()
selected_rides.add(N)

total_t = 0
total_reward = 0
total_bonus = 0
for i_vcl in tqdm(range(F)):
    i_cur_ride = N
    cur_t = 0
    cur_ride = rides[i_cur_ride]    
    while True:
        max_reward = -np.inf
        max_i_ride_follow = None
        max_bonus = None
        max_real_reward = None
        max_waiting_time = None
        max_dist = None
        max_end2start = None
        for i_ride_follow in R_links[str(i_cur_ride)]:
            if i_ride_follow in selected_rides: continue
            ride_follow = rides[i_ride_follow]
            end2start = End2Start[i_cur_ride, i_ride_follow] 
            if not is_possible(ride_follow, end2start, cur_t): continue
            bonus, real_reward, waiting_time, dist, reward = cal_reward(cur_ride, ride_follow, cur_t, end2start)
            if reward > max_reward:
                max_reward = reward
                max_real_reward = real_reward
                max_i_ride_follow = i_ride_follow
                max_bonus = bonus
                max_waiting_time = waiting_time
                max_dist = dist
        if max_i_ride_follow is None: break
        total_reward += max_real_reward
        total_bonus += max_bonus
        
        delta_t = max_waiting_time + max_dist
        cur_t += delta_t
        total_t += delta_t
        
        #coeff_waiting_loss = coeff_waiting_loss*coeff_exp + (1-coeff_exp)*(max_real_reward / delta_t)
        #coeff_bonus_loss = coeff_bonus_loss*coeff_exp + (1-coeff_exp)*(max_bonus / delta_t)
        
        coeff_waiting_loss = total_reward / total_t
        coeff_bonus_loss = total_bonus / total_t
        
        i_cur_ride = max_i_ride_follow
        cur_ride = rides[i_cur_ride]
        
        VclRides[i_vcl].append(i_cur_ride)
        selected_rides.add(i_cur_ride)

print (f'Total Reward: {total_reward}')

In [97]:
with open(fout, 'w') as f:
    for i, vcl in enumerate(VclRides):
        ride_ls = ' '.join([str(i_ride) for i_ride in vcl])
        f.write(f'{len(vcl)} {ride_ls} \n')

In [12]:
# Bread First Search instead of Deep First Search
coeff_waiting_loss = (B + avg_dist)/(avg_dist + avg_end2start)
coeff_bonus_loss = B / (avg_dist + avg_end2start)
coeff_exp = 0.80

VclRides = [list() for _ in range(F)]
selected_rides = set()
selected_rides.add(N)

total_t = 0
total_reward = 0
total_bonus = 0
VclCurrRide = [N for _ in range(F)]
VclCurrT = [0 for _ in range(F)]
VclStopped = [False for _ in range(F)]

while True:
    stop = True
    for i_vcl in tqdm(range(F)):
        if VclStopped[i_vcl]: continue
            
        i_cur_ride = VclCurrRide[i_vcl]
        cur_ride = rides[i_cur_ride]
        cur_t = VclCurrT[i_vcl]
        
        max_reward = -np.inf
        max_i_ride_follow = None
        max_bonus = None
        max_real_reward = None
        max_waiting_time = None
        max_dist = None
        for i_ride_follow in R_links[str(i_cur_ride)]:
            if i_ride_follow in selected_rides: continue
            ride_follow = rides[i_ride_follow]
            end2start = End2Start[i_cur_ride, i_ride_follow] 
            if not is_possible(ride_follow, end2start, cur_t): continue
            bonus, real_reward, waiting_time, dist, reward = cal_reward(cur_ride, ride_follow, cur_t, end2start)
            if reward > max_reward:
                max_reward = reward
                max_real_reward = real_reward
                max_i_ride_follow = i_ride_follow
                max_bonus = bonus
                max_waiting_time = waiting_time
                max_dist = dist
                
        if max_i_ride_follow is None: 
            VclStopped[i_vcl] = True
            continue
            
        #if max_i_ride_follow == 2433:
        #    import pdb; pdb.set_trace()   
        stop = False
        
        total_reward += max_real_reward
        total_bonus += max_bonus

        delta_t = max_waiting_time + max_dist
        
                    
        cur_t += delta_t
        total_t += delta_t
            
        #coeff_waiting_loss = coeff_waiting_loss*coeff_exp + (1-coeff_exp)*(max_real_reward / delta_t)
        #coeff_bonus_loss = coeff_bonus_loss*coeff_exp + (1-coeff_exp)*(max_bonus / delta_t)
        coeff_waiting_loss = total_reward / total_t
        coeff_bonus_loss = total_bonus / total_t

        i_cur_ride = max_i_ride_follow
        VclCurrT[i_vcl] = cur_t
        VclRides[i_vcl].append(i_cur_ride)
        VclCurrRide[i_vcl] = i_cur_ride
        selected_rides.add(i_cur_ride)
        
    if stop: break
print (f'Total Reward: {total_reward}')

 59%|█████▉    | 207/350 [00:07<00:04, 28.89it/s]


KeyboardInterrupt: 

In [ ]:
reward = 0
for i, vcl in enumerate(best_vclRides):
    cur_t = 0
    i_cur_ride = N
    for i_ride in vcl:
        begin_cur_t = cur_t
        cur_t = cur_t + End2Start[i_cur_ride, i_ride]
        ride = rides[i_ride]
        if cur_t <= ride['start_els']: 
            cur_t = ride['start_els']
            reward += B
        cur_t += ride['distance']
        if cur_t >= ride['finish']: 
            import pdb; pdb.set_trace()
            print("Invalid!")
        reward += ride['distance']
        i_cur_ride = i_ride

In [113]:
import random
random.seed(80)
# Bread First Search and DFS together
def cal_loss_waiting_time(waiting_time):
    return waiting_time * vcl_coeff_waiting_loss

def cal_loss_bonus(distance):
    return distance * vcl_coeff_bonus_loss

coeff_waiting_loss = (B + avg_dist)/(avg_dist + avg_end2start)
coeff_bonus_loss = B / (avg_dist + avg_end2start)

VclRides = [list() for _ in range(F)]

selected_rides = set()
selected_rides.add(N)

total_t = 0
total_reward = 0
total_bonus = 0
VclCurrRide = [N for _ in range(F)]
VclCurrT = [0 for _ in range(F)]
VclStopped = [False for _ in range(F)]

VclWaitingCoeff = [2+2*random.random() for _ in range(F)] #Good for d - (2, 2)
VclBonusCoeff = [2+2*random.random() for _ in range(F)]

while True:
    stop = True
    for i_vcl in tqdm(range(F)):
        if VclStopped[i_vcl]: continue
        n_steps = 500
        vcl_coeff_waiting_loss = coeff_waiting_loss #* VclWaitingCoeff[i_vcl]
        vcl_coeff_bonus_loss = coeff_bonus_loss #* VclWaitingCoeff[i_vcl]
        while n_steps > 0:
            i_cur_ride = VclCurrRide[i_vcl]
            cur_ride = rides[i_cur_ride]
            cur_t = VclCurrT[i_vcl]

            max_reward = -np.inf
            max_i_ride_follow = None
            max_bonus = None
            max_real_reward = None
            max_waiting_time = None
            max_dist = None
            for i_ride_follow in R_links[str(i_cur_ride)]:
                if i_ride_follow in selected_rides: continue
                ride_follow = rides[i_ride_follow]
                end2start = End2Start[i_cur_ride, i_ride_follow] 
                if not is_possible(ride_follow, end2start, cur_t): continue
                bonus, real_reward, waiting_time, dist, reward = cal_reward(cur_ride, ride_follow, cur_t, end2start)
                #if dist > avg_dist*5:
                #    reward = reward*0.5
                if reward > max_reward:
                    max_reward = reward
                    max_real_reward = real_reward
                    max_i_ride_follow = i_ride_follow
                    max_bonus = bonus
                    max_waiting_time = waiting_time
                    max_dist = dist

            if max_i_ride_follow is None: 
                VclStopped[i_vcl] = True
                break

            #if max_i_ride_follow == 2433:
            #    import pdb; pdb.set_trace()   
            stop = False

            total_reward += max_real_reward
            total_bonus += max_bonus

            delta_t = max_waiting_time + max_dist


            cur_t += delta_t
            total_t += delta_t

            #coeff_waiting_loss = coeff_waiting_loss*coeff_exp + (1-coeff_exp)*(max_real_reward / delta_t)
            #coeff_bonus_loss = coeff_bonus_loss*coeff_exp + (1-coeff_exp)*(max_bonus / delta_t)
            coeff_waiting_loss = total_reward / total_t
            coeff_bonus_loss = total_bonus / total_t

            i_cur_ride = max_i_ride_follow
            VclCurrT[i_vcl] = cur_t
            VclRides[i_vcl].append(i_cur_ride)
            VclCurrRide[i_vcl] = i_cur_ride
            selected_rides.add(i_cur_ride)
            n_steps -= 1
        
    if stop: break
print (f'Total Reward: {total_reward}')

100%|██████████| 400/400 [00:00<00:00, 2979967.32it/s]

Total Reward: 9952254


In [27]:
random.random()

0.29805650321609556